In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from os import listdir
import numpy as np
import math
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist
from scipy.spatial import distance
from itertools import groupby

In [2]:
def calculateMass(data):
#      multiply by constant 997
    result = 0
    for d in data:
        result += (1/6)*math.pi*math.pow(d,3) * 917 * math.pow(10, 6)
#     return round(result, 3)
    return result

In [5]:
def calculateLength(data):
    """
    find the most distant point
    data = a nupmpy array containing the points
    """
    """first check if the x points are same or not
    if same we will use the naive techniques
    if not we will use the convex hull approach
    """
    xVals = [i[0] for i in data]
#     print(xVals)
    g = groupby(xVals)
    sameValues = next(g, True) and not next(g, False)
#     print(sameValues)
    
    if sameValues == False:
        # # Find a convex hull in O(N log N)
        hull = ConvexHull(data)

        # Extract the points forming the hull
        hullpoints = data[hull.vertices,:]
    
    elif sameValues == True:
        hullpoints = data       

#     print(hullpoints)
    
    # Naive way of finding the best pair in O(H^2) time if H is number of points on
    # hull
    hdist = cdist(hullpoints, hullpoints, metric='euclidean')

    # Get the farthest apart points
    bestpair = np.unravel_index(hdist.argmax(), hdist.shape)

    #Print them
    points = [hullpoints[bestpair[0]],hullpoints[bestpair[1]]]
#     print(points)

    # print the distance
#     dist = np.linalg.norm(points[0] - points[1])
    dst = distance.euclidean(points[0], points[1])
    print(points[0], points[1], dst)
    return round(dst, 3)

In [6]:
columns = ["cluster", "nodeCluster", "temperature", "ice_particles", "ice_mass", "ice_length"]
# folders = [f for f in listdir("data/withOutliers/")]
folders = ['newData']
for folder in folders:
    onlyfiles = [f for f in listdir("data/withOutliers/{}".format(folder)) ]
#     print(folder)
    for file in onlyfiles:
        path = "data/withOutliers/{}/{}".format(folder, file)
#         print(path)
        df = pd.read_csv(path)
#             calculating the diameter in micron (?)
#         df['new_d'] = df['d'] * math.pow(10, 6)
    
#         getting the temperature means
        means = round(df.groupby('cluster')['T'].mean(), 3)
#         print(df.groupby('cluster')['nodeCluster'])
#     getting the clusters
        clusters = means.index.tolist()
#         print(clusters)
#     getting the nodeCluster
        nodeClusters = sorted(df['nodeCluster'].value_counts().index.tolist())
#         print(nodeClusters)
#     storing the temperature
        temperature = means.tolist()
#     storing total particles
        particles = df.groupby('cluster')['cluster'].value_counts().tolist()

#     adding the total values for all data
        clusters.append("all")
        nodeClusters.append('all')
        temperature.append(round(df['T'].mean(), 3))
        particles.append(df.shape[0])

#         extracting the total volumes
#         volumes = []
        volumesExact = []
#         extracting the farthest distance
        distances = []
        for each in clusters:
#             print(each)
            if each == 'all':
                print(each)
#                 volumes.append(calculateVolume(df['new_d'].tolist()))
                volumesExact.append(calculateMass(df['d'].tolist()))
                distances.append(calculateLength(df[['Points:0', 'Points:1', 'Points:2']].to_numpy()))
            else:
                print(each)
#                 volumes.append(calculateVolume(df[df.cluster == each]['new_d'].tolist()))
                volumesExact.append(calculateMass(df[df.cluster == each]['d'].tolist()))
                distances.append(calculateLength(df[df.cluster == each][['Points:0', 'Points:1', 'Points:2']].to_numpy()))
        
        
        # print(means)
#         print(clusters)
#         print(temperature)
#         print(particles)
#         print(volumes)
#         print(distances)
#         print(volumesExact)

# output csv generation
        extract = pd.DataFrame(np.column_stack([clusters, nodeClusters, temperature, particles, volumesExact, distances]),
                      columns=columns)

        extract.to_csv('output/withOutliers/{}/{}'.format(folder, file), index=False)
        print("exported {} {} ".format(folder, file))


2
[2.9872e+01 5.3981e-01 2.5889e-10] [ 5.29590000e+01 -1.19520000e+00 -4.06809985e-09] 23.152102036318425
3
[4.8062e+00 3.8889e-01 5.9698e-12] [ 5.13180000e+01 -2.17250000e+00 -9.48789973e-09] 46.58227407471754
4
[ 2.3460e+01  6.4640e-01 -1.2554e-09] [ 2.73620000e+01 -5.66810000e-01 -4.08289993e-09] 4.086255315579288
5
[28.692       0.51454505  0.61321092] [28.692      -0.51454505 -0.61321092] 1.60098
6
[13.596      -0.3978222   0.33381246] [12.729       0.33889806 -0.28436924] 1.2948337184364622
7
[2.2820e+01 6.3302e-01 3.8808e-09] [ 2.17380000e+01 -6.31610000e-01  3.53710773e-11] 1.6643356142617372
all
[4.8062e+00 3.8889e-01 5.9698e-12] [ 5.29590000e+01 -1.19520000e+00 -4.06809985e-09] 48.17884897927824
exported newData 210.csv 
2
[2.8126e+01 4.5802e-01 5.7741e-10] [ 5.29460000e+01 -2.07240000e-01 -1.28949997e-09] 24.828914009025862
3
[13.609      -0.52141664  0.18978014] [51.086       2.21870825 -0.80754375] 37.59027092693535
4
[ 2.2026e+01  6.0696e-01 -1.4855e-12] [ 2.05950000e+01 

[ 8.6486e+00  4.3233e-01 -1.4153e-07] [ 8.6486e+00 -4.3233e-01  1.4153e-07] 0.8646600000000463
12
[12.03        0.4003198   0.23112576] [12.03       -0.4003198  -0.23112576] 0.9245000000016793
all
[ 3.7374e+00  3.6334e-01 -6.7589e-06] [ 5.2798e+01 -6.4441e-01 -4.5048e-07] 49.0709489659874
exported newData 219.csv 


In [10]:
path = "data/withoutOutliers/{}/{}".format("contrails1", "0.2.csv")
df = pd.read_csv(path)
pts = df[['Points:0', 'Points:1', 'Points:2']].to_numpy()
hull = ConvexHull(p)

In [14]:

from mpl_toolkits.mplot3d import Axes3D


# 8 points defining the cube corners
# pts = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],
#                 [0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1], ])

hull = ConvexHull(pts)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")

# # Plot defining corner points
# ax.plot(pts.T[0], pts.T[1], pts.T[2], "ko")

# # 12 = 2 * 6 faces are the simplices (2 simplices per square face)
# for s in hull.simplices:
#     s = np.append(s, s[0])  # Here we cycle back to the first coordinate
#     ax.plot(pts[s, 0], pts[s, 1], pts[s, 2], "r-")

# # Make axis label
# for i in ["x", "y", "z"]:
#     eval("ax.set_{:s}label('{:s}')".format(i, i))

# plt.show()

print(hull)

In [7]:
x = rand(1,20);              # 20 random x values
y = rand(1,20);              # 20 random y values
hullPoints = convhull(x,y);  # Find the points defining the convex hull
patch(x(hullPoints),y(hullPoints),'r');  # Plot the convex hull in red
hold on;                     # Add to the existing plot
scatter(x,y);    

SyntaxError: invalid syntax (Temp/ipykernel_7936/3301890463.py, line 5)

In [8]:
[4.8062e+00 3.8889e-01 5.9698e-12] [ 5.29590000e+01 -1.19520000e+00 -4.06809985e-09] 48.17884897927824

SyntaxError: invalid syntax (Temp/ipykernel_7936/1113393829.py, line 1)

In [11]:
5.29590000e+01

52.959